# Загрузка Pandas и очистка данных

In [129]:
import pandas as pd
import numpy as np
import json

In [80]:
original_df = pd.read_csv('data/main_task.csv')


In [82]:
print(original_df.info())
original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [83]:
# На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типа object, и заполним пропущенные значения (None или NaN) каким-то одним значением (нулём или средним арифметическим) для  всего столбца.

object_cols = list(original_df.select_dtypes(include=['object']).columns)
df = original_df.drop(labels=object_cols, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  37457 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [84]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [85]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [86]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [87]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [88]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [89]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

NameError: name 'y_pred' is not defined

Для упрощения тестирования объеденим код в функцию принимающую датафрейм и возвращающую значение переменной MAE.

In [90]:
def calc_mae(df):
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
    X = df.drop(['Rating'], axis = 1)
    y = df['Rating']
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
    # Для тестирования мы будем использовать 25% от исходного датасета.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    # Создаём модель
    regr = RandomForestRegressor(n_estimators=100)
    # Обучаем модель на тестовом наборе данных
    regr.fit(X_train, y_train)
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
    # Предсказанные значения записываем в переменную y_pred
    y_pred = regr.predict(X_test)
    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
    # Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
    return metrics.mean_absolute_error(y_test, y_pred)

Посчитаем значение MAE для замены пропусков разными статистиками:

In [91]:
filler = 0
print('0:\t', calc_mae(df.fillna(filler)))
filler = df['Number of Reviews'].mean()
print('mean = ', filler)
print('mean:\t', calc_mae(df.fillna(filler)))
filler = df['Number of Reviews'].median()
print('median = ', filler)
print('median:\t', calc_mae(df.fillna(filler)))
modes = df['Number of Reviews'].mode().values
print('modes: ', modes)
for mode in modes:
    filler = mode
    print('mode = ', filler)
    print('mode:\t', calc_mae(df.fillna(filler)))

0:	 0.4385521448412698
mean =  124.82547988359985
mean:	 0.43264359893578647
median =  33.0
median:	 0.43444806369047617
modes:  [2.]
mode =  2.0
mode:	 0.4366378793650793


## С одной стороны, всё просто. Задачу, которая стоит перед вами, можно свести к трём пунктам:

Удалить из датафрейма столбцы, данные в которых представлены не числами (это вы уже сделали, и нужно просто повторить знакомые действия, но в этот раз выполнить данный шаг в последнюю очередь).
Избавиться от пропущенных (None) значений (на предыдущем шаге мы делали это самым грубым из всех возможных способов; сейчас попробуем подойти к процессу более гибко).
Создать новые столбцы с данными, используя для этого информацию, содержащуюся в других столбцах датафрейма (например, можно добавить столбец, сообщающий, сколько дней прошло со дня публикации последнего отзыва, отображённого на сайте).
С другой стороны, в этом задании масса подводных камней.


## Строковые данные
В исходном наборе данных всего три столбца содержат числовые данные, причём один из этих столбцов — это целевая переменная, значение которой должна предсказывать наша модель. Так что просто удалить все object-значения и считать задачу выполненной не получится. Для создания качественной модели нам сначала придётся очень основательно поработать со строковыми данными и извлечь из них как можно больше информации, которую можно представить в числовом виде.


## Пропущенные значения
Мы уже говорили о том, что в задачах по машинному обучению принято не удалять строки с пустыми значениями, а заполнять их максимально близкими к реальности данными. Как найти такие данные — большой вопрос, требующий не только знания синтаксиса, но и креативности, изобретательности, хорошего понимания контента, а иногда ещё и интуиции. 


## Новые признаки
Это, пожалуй, самая сложная, но и самая интригующая, увлекательная и творческая часть работы на данном шаге. Создание новых признаков (Feature Engineering) потребует от вас не только хорошего владения разными библиотеками Python, но и способность вникать в контент, умение подключать к работе интуицию, творческий подход и готовность к экспериментам.

Для удобства нормальзуем названия колонок, приведем их к snake_case.

In [106]:
df = pd.read_csv('data/main_task.csv')
df.columns = [col.lower().replace(' ', '_') for col in list(original_df.columns.values)]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


### Вопросы о ценах
Сколько вариантов непустых значений встречается в столбце Price Range?

In [107]:
df.price_range.value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: price_range, dtype: int64

price_range - явно ординальный признак, заменим значения на следующие:
* `$` = 1
* `$$ - $$$` = 2
* `$$$$` = 3

In [110]:
def nomalize_price_range(value):
    if value == '$':
        return 1
    elif value == '$$ - $$$':
        return 2
    elif value == '$$$$':
        return 3
    return value

df.price_range = df.price_range.apply(nomalize_price_range)
df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


## Вопрос о городах

In [113]:
len(df.city.unique())

31

## Вопросы о кухнях
Сколько типов кухонь представлено в наборе данных?

In [156]:
def jsonify_string(str):
    return '{"list":' + str.replace("'", '"') + '}'

def string_to_list(str):
    return str[1:-1].replace("'", "").split(', ')

cousines = set()
df.cuisine_style.dropna().apply(lambda c: cousines.update(string_to_list(c)))
print(len(cousines))


125


Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.


In [158]:
cousines_popularity = {}
most_popular = None
for cousine in cousines:
    cousines_popularity[cousine] = 0

print(cousines_popularity)

{'Romanian': 0, 'Ecuadorean': 0, 'Vegan Options': 0, 'Delicatessen': 0, 'Vegetarian Friendly': 0, 'Georgian': 0, 'Israeli': 0, 'Latvian': 0, 'Polynesian': 0, 'Turkish': 0, 'Filipino': 0, 'Taiwanese': 0, 'Bar': 0, 'Grill': 0, 'Cafe': 0, 'Latin': 0, 'Mexican': 0, 'German': 0, 'Balti': 0, 'Halal': 0, 'Cajun & Creole': 0, 'Ukrainian': 0, 'British': 0, 'Malaysian': 0, 'Argentinean': 0, 'Arabic': 0, 'Gastropub': 0, 'Yunnan': 0, 'New Zealand': 0, 'African': 0, 'Welsh': 0, 'Central American': 0, 'Chinese': 0, 'Barbecue': 0, 'Singaporean': 0, 'Jamaican': 0, 'Cambodian': 0, 'Middle Eastern': 0, 'Croatian': 0, 'Burmese': 0, 'Armenian': 0, 'Norwegian': 0, 'Sri Lankan': 0, 'Seafood': 0, 'Tunisian': 0, 'Southwestern': 0, 'Indian': 0, 'Central Asian': 0, 'International': 0, 'Asian': 0, 'Portuguese': 0, 'Afghani': 0, 'Tibetan': 0, 'Belgian': 0, 'Wine Bar': 0, 'Brazilian': 0, 'Indonesian': 0, 'Eastern European': 0, 'Japanese': 0, 'Gluten Free Options': 0, 'Steakhouse': 0, 'Nepali': 0, 'Moroccan': 0, 'F

Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [ ]:
df['cuisine_style_count'] = df.cuisine_style.dropna().apply(lambda c: len(json.loads(jsonify_string(c))['list']))
df.head()